In [1]:
!pip install langchain langchain_openai langchain_community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [1]:
from dotenv import load_dotenv

load_dotenv()


True

## Loader

In [2]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 경로 리스트
pdf_files = [
    "2024년 하반기 3급 신입사원 채용 직무소개서.pdf",
    "삼성전자 DS부문 24하 공채 직무기술서.pdf"
]

# 모든 문서 로드
docs = []
for path in pdf_files:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

# 총 로드된 문서 수 확인
print("총 문서 수:", len(docs))


총 문서 수: 188


## Splitter

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=0)

In [5]:
split_doc1 = text_splitter.split_documents(docs)
len(split_doc1)

374

In [6]:
split_doc1[0]

Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2024-09-04T13:46:13+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'admin', 'moddate': '2024-09-04T13:46:13+09:00', 'source': '2024년 하반기 3급 신입사원 채용 직무소개서.pdf', 'total_pages': 84, 'page': 1, 'page_label': '2'}, page_content='회로개발\n회로 기술을 기반으로 삼성전자 제품 및 솔루션의 혁신적인 가치를 창출합니다')

## Embedding model

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

## VectorStore 생성
- FAISS는 고성능 벡터 검색 및 클러스터링을 위한 라이브러리입니다.
- 이 클래스는 FAISS를 LangChain의 VectorStore 인터페이스와 통합합니다.
- 임베딩 함수, FAISS 인덱스, 문서 저장소를 조합하여 효율적인 벡터 검색 시스템을 구축할 수 있습니다.

In [8]:
import faiss
from langchain_community.vectorstores import FAISS

### from_documents()
- `from_documents` 클래스 메서드는 문서 리스트와 임베딩 함수를 사용하여 FAISS 벡터 저장소를 생성합니다.
- 매개변수
  - `documents (List[Document])`: 벡터 저장소에 추가할 문서 리스트
  - `embedding (Embeddings)`: 사용할 임베딩 함수
  - `**kwargs`: 추가 키워드 인자

In [9]:
split_doc1[0]

Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2024-09-04T13:46:13+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'admin', 'moddate': '2024-09-04T13:46:13+09:00', 'source': '2024년 하반기 3급 신입사원 채용 직무소개서.pdf', 'total_pages': 84, 'page': 1, 'page_label': '2'}, page_content='회로개발\n회로 기술을 기반으로 삼성전자 제품 및 솔루션의 혁신적인 가치를 창출합니다')

In [10]:
# DB 생성
db_docs = FAISS.from_documents(
    documents=split_doc1, embedding=embeddings)

In [11]:
len(split_doc1)

374

In [ ]:
db_docs.index_to_docstore_id

{0: '49646d52-5538-4b10-ba66-262883ef9d49',
 1: 'cb52cc84-2e60-4893-9df2-fe76734df6fc',
 2: 'f58efd82-8474-4505-92ef-7b0e14e8c82e',
 3: '6d7931dc-7de4-4439-9515-26e265222fa2',
 4: '7d3598ea-3be7-4ea8-bc4c-56654832c2ab',
 5: '8bbdd53c-74dc-416f-97cd-16e59a861130',
 6: '0d1cdec0-c4d4-4e3e-81c8-9c4590683a2f',
 7: 'b1f36e94-20f7-4af0-a01d-64fed31b057f',
 8: 'b1f4664d-9836-4773-bf17-72f6df45c05f',
 9: '1c38d7ce-7325-4221-8233-c42befad0c23',
 10: '7da4b39b-fdfb-4c36-a62f-069381a1fb14',
 11: 'f32d2213-4958-4777-954d-7c0b97d8ae7a',
 12: 'c901d3f1-681b-455e-a451-1820ac624369',
 13: 'd51a5dca-05e6-4c35-9a93-9e0131f27284',
 14: 'a2b84df1-c0ef-4363-acd4-a186dc284e37',
 15: '4deff71d-65a4-469e-820c-025b5616b2d7',
 16: '67b4f4ec-4105-402f-8028-648217113082',
 17: 'a9b976d1-104e-478c-ba4b-84a65bfa3801',
 18: '643ee865-848c-4c5a-98bc-e036abd4bd37',
 19: 'efbec5d5-b942-4cd1-8507-28c54fef39ed',
 20: '7c21f429-6088-4f41-b3e5-443cd95c58d6',
 21: 'a789b45b-782e-4306-9e2e-eedc6e774114',
 22: '38e23c05-560e-

In [13]:
db_docs.save_local(folder_path="faiss_db", index_name="faiss_index")

### 로컬에서 불러오기 (Load Local)
- load_local 클래스 메서드는 로컬 디스크에 저장된 FAISS 인덱스, 문서 저장소, 그리고 인덱스-문서 ID 매핑을 불러오는 기능을 제공합니다.
- 매개변수
  - `folder_path (str)`: 불러올 파일들이 저장된 폴더 경로
  - `embeddings (Embeddings)`: 쿼리 생성에 사용할 임베딩 객체
  - `index_name (str)`: 불러올 인덱스 파일 이름 (기본값: "index")
  - `allow_dangerous_deserialization (bool)`: pickle 파일 역직렬화 허용 여부 (기본값: False)

In [36]:
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

In [37]:
# 로드된 데이터를 확인
loaded_db.index_to_docstore_id

{0: '6783edc9-3e30-4ec5-b40c-b1da201560d1',
 1: '62e40f73-aa20-4a1c-9dda-78a558008ca7',
 2: '8687eae6-c22c-4b16-9465-71830fe28856',
 3: '29129a97-3992-42f0-a639-8fc72498f203',
 4: 'cbe02339-29e8-447c-9f42-e8431c6384f5',
 5: 'd9e83fac-0b4c-41bc-9424-2f285142dc0b',
 6: 'd6d0264d-7018-44b9-932a-0977a7ba01de',
 7: '46df286d-65b0-4d38-950f-bd1b51680240',
 8: '897c840e-45d8-4df8-9f9b-0bcaafd2932e',
 9: '3c712e5c-f0c3-4b0b-97b5-c5a7b32708cb',
 10: 'ad0437c1-a33c-4ab9-9811-b76f766ea950'}

In [38]:
db.docstore._dict[db.index_to_docstore_id[1]]

Document(id='62e40f73-aa20-4a1c-9dda-78a558008ca7', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nVectorStore\n\n정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.\n예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.\n연관키워드: 임베딩, 데이터베이스, 벡터화\n\nSQL\n\n정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다. 데이터 조회, 수정, 삽입, 삭제 등 다양한 작업을 수행할 수 있습니다.\n예시: SELECT * FROM users WHERE age > 18;은 18세 이상의 사용자 정보를 조회합니다.\n연관키워드: 데이터베이스, 쿼리, 데이터 관리\n\nCSV')

## 검색

### 유사도 검색 (Similarity Search)
- `similarity_search` 메서드는 주어진 쿼리와 가장 유사한 문서들을 검색하는 기능을 제공합니다.
- 매개변수
  - `query (str)`: 유사한 문서를 찾기 위한 검색 쿼리 텍스트
  - `k (int)`: 반환할 문서 수. 기본값은 4
  - `filter (Optional[Union[Callable, Dict[str, Any]]])`: 메타데이터 필터링 함수 또는 딕셔너리. 기본값은 None
  - `fetch_k (int)`: 필터링 전에 가져올 문서 수. 기본값은 20
  - `**kwargs`: 추가 키워드 인자

In [39]:
db = FAISS.from_documents(
    documents=split_doc1, embedding=embeddings)

In [40]:
db.index_to_docstore_id

{0: 'a8840c4a-38f4-4008-a437-070a84cecaa9',
 1: '9c417992-3872-4648-94d1-5a230b9bd02f',
 2: '63a684c6-cbf0-4f04-ad05-5d26a60f76fd',
 3: '48abff8e-256a-4857-9c2f-e5738c220f29',
 4: '3f56b9d5-bc70-4b8d-925a-dac6a2e3e28d',
 5: 'cca72bda-c169-4251-86ee-c7e700b4a473',
 6: 'f61cb925-e2e1-4c59-a51a-484b5e00ca41',
 7: '2630dc6b-55b5-4bcc-b609-c9df8c63e80f',
 8: '6a587118-15ce-43d0-9090-7cde064d1e61',
 9: '1fd1fc36-2fb5-4622-ad11-66fb600fd4bb',
 10: 'e0f51d87-0f7a-4c7f-afd0-39bfcdb8f922'}

In [41]:
db.docstore._dict[db.index_to_docstore_id[0]]

Document(id='a8840c4a-38f4-4008-a437-070a84cecaa9', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nTokenizer')

#### 유사도 검색

In [42]:
# 유사도 검색
result = db.similarity_search("TF IDF 에 대하여 알려줘")
len(result)

4

In [43]:
result[0]

Document(id='f61cb925-e2e1-4c59-a51a-484b5e00ca41', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame')

In [44]:
result[-1]

Document(id='6a587118-15ce-43d0-9090-7cde064d1e61', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍스트 생성, 딥러닝\n\nInstructGPT\n\n정의: InstructGPT는 사용자의 지시에 따라 특정한 작업을 수행하기 위해 최적화된 GPT 모델입니다. 이 모델은 보다 정확하고 관련성 높은 결과를 생성하도록 설계되었습니다.\n예시: 사용자가 "이메일 초안 작성"과 같은 특정 지시를 제공하면, InstructGPT는 관련 내용을 기반으로 이메일을 작성합니다.\n연관키워드: 인공지능, 자연어 이해, 명령 기반 처리\n\nKeyword Search')

#### k 값 지정

In [45]:
# k 값 지정
result = db.similarity_search("TF IDF 에 대하여 알려줘", k=2)
len(result)

2

In [46]:
result

[Document(id='f61cb925-e2e1-4c59-a51a-484b5e00ca41', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='cca72bda-c169-4251-86ee-c7e700b4a473', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시

#### filter 사용

In [47]:
# filter 사용
result = db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": DATA_PATH+"nlp-keywords.txt"}, k=2
)
len(result)

2

In [48]:
result

[Document(id='f61cb925-e2e1-4c59-a51a-484b5e00ca41', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='cca72bda-c169-4251-86ee-c7e700b4a473', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시

### [최대 한계 관련성(Maximum Marginal Relevance, MMR)](https://wikidocs.net/231585)
- 정보 검색이나 문서 요약에서 사용되는 알고리즘으로, 문서의 관련성과 다양성을 동시에 고려하는 방법.
- 선택된 문서가 질의(query)에 얼마나 관련이 있는지(Relevance), 그리고 이미 선택된 문서들과 얼마나 다른지(Diversity)를 계산하여 최적의 문서를 선정.
- MMR 알고리즘은 질의에 대한 관련성이 높은 문서를 찾으면서도, 이미 선택된 문서들과는 상이한 내용을 가진 문서를 선호함으로써, 결과의 다양성을 보장함.

예를 들어 사용자가 주제에 대해 폭넓은 정보를 얻고자 할 때 유용하며, 모든 문서가 같은 내용을 반복하지 않도록 하는 데에 목적이 있음.

MMR의 주요 매개변수:
- `query`: 사용자로부터 입력받은 검색 쿼리입니다.
- `k`: 최종적으로 선택할 문서의 수입니다. 이 매개변수는 반환할 문서의 총 개수를 결정합니다.
- `fetch_k`: MMR 알고리즘을 수행할 때 고려할 상위 문서의 수입니다. 이는 초기 후보 문서 집합의 크기를 의미하며, 이 중에서 MMR에 의해 최종 문서가 k개 만큼 선택됩니다.
- `lambda_mult`: 쿼리와의 유사성과 선택된 문서 간의 다양성 사이의 균형을 조절합니다. $(\lambda = 1)$은 유사성만 고려하며, $(\lambda = 0)$은 다양성만을 최대화합니다.

In [49]:
result = db.max_marginal_relevance_search(
    "TF IDF 에 대하여 알려줘", k=4, fetch_k=10)

In [50]:
len(result)

4

In [51]:
result[0]

Document(id='f61cb925-e2e1-4c59-a51a-484b5e00ca41', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame')

In [52]:
result[-1]

Document(id='63a684c6-cbf0-4f04-ad05-5d26a60f76fd', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='정의: CSV(Comma-Separated Values)는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다. 표 형태의 데이터를 간단하게 저장하고 교환할 때 사용됩니다.\n예시: 이름, 나이, 직업이라는 헤더를 가진 CSV 파일에는 홍길동, 30, 개발자와 같은 데이터가 포함될 수 있습니다.\n연관키워드: 데이터 형식, 파일 처리, 데이터 교환\n\nJSON\n\n정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nHuggingFace')

## FAISS 객체 병합 (Merge From)
- merge_from 메서드는 현재 FAISS 객체에 다른 FAISS 객체를 병합하는 기능을 제공합니다.
- 매개변수
  - `target (FAISS)`: 현재 객체에 병합할 대상 FAISS 객체

In [53]:
# db 의 데이터 확인
db.index_to_docstore_id

{0: 'a8840c4a-38f4-4008-a437-070a84cecaa9',
 1: '9c417992-3872-4648-94d1-5a230b9bd02f',
 2: '63a684c6-cbf0-4f04-ad05-5d26a60f76fd',
 3: '48abff8e-256a-4857-9c2f-e5738c220f29',
 4: '3f56b9d5-bc70-4b8d-925a-dac6a2e3e28d',
 5: 'cca72bda-c169-4251-86ee-c7e700b4a473',
 6: 'f61cb925-e2e1-4c59-a51a-484b5e00ca41',
 7: '2630dc6b-55b5-4bcc-b609-c9df8c63e80f',
 8: '6a587118-15ce-43d0-9090-7cde064d1e61',
 9: '1fd1fc36-2fb5-4622-ad11-66fb600fd4bb',
 10: 'e0f51d87-0f7a-4c7f-afd0-39bfcdb8f922'}

### db2 생성

In [54]:
loader2 = TextLoader(DATA_PATH+"finance-keywords.txt")

In [55]:
split_doc2 = loader2.load_and_split(text_splitter)
len(split_doc2)

6

In [56]:
# 새로운 FAISS 벡터 저장소 생성
db2 = FAISS.from_documents(documents=split_doc2, embedding=OpenAIEmbeddings())

In [57]:
# db2 의 데이터 확인
db2.index_to_docstore_id

{0: '02f529e9-9e01-44a5-82d0-8a347f7c94a6',
 1: '00e9e84c-78db-4f1f-b33a-72d07f40ca0c',
 2: 'b4de7c85-db3f-47aa-ba6e-33d4f78e3179',
 3: '4bf38b9e-492b-46c5-b9ed-0102e1b9e074',
 4: '6439eda7-2532-4171-9ded-c768b697dfae',
 5: '4c00b4d2-a25b-4691-a3ea-0f476ed0a058'}

### 병합

In [58]:
# db + db2 를 병합
db.merge_from(db2)

In [59]:
# 병합된 데이터 확인
db.index_to_docstore_id

{0: 'a8840c4a-38f4-4008-a437-070a84cecaa9',
 1: '9c417992-3872-4648-94d1-5a230b9bd02f',
 2: '63a684c6-cbf0-4f04-ad05-5d26a60f76fd',
 3: '48abff8e-256a-4857-9c2f-e5738c220f29',
 4: '3f56b9d5-bc70-4b8d-925a-dac6a2e3e28d',
 5: 'cca72bda-c169-4251-86ee-c7e700b4a473',
 6: 'f61cb925-e2e1-4c59-a51a-484b5e00ca41',
 7: '2630dc6b-55b5-4bcc-b609-c9df8c63e80f',
 8: '6a587118-15ce-43d0-9090-7cde064d1e61',
 9: '1fd1fc36-2fb5-4622-ad11-66fb600fd4bb',
 10: 'e0f51d87-0f7a-4c7f-afd0-39bfcdb8f922',
 11: '02f529e9-9e01-44a5-82d0-8a347f7c94a6',
 12: '00e9e84c-78db-4f1f-b33a-72d07f40ca0c',
 13: 'b4de7c85-db3f-47aa-ba6e-33d4f78e3179',
 14: '4bf38b9e-492b-46c5-b9ed-0102e1b9e074',
 15: '6439eda7-2532-4171-9ded-c768b697dfae',
 16: '4c00b4d2-a25b-4691-a3ea-0f476ed0a058'}

# FAISS with Retriever

## Loader

In [60]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

loader = DirectoryLoader(path=DATA_PATH, glob='*.txt'
                    , show_progress=True, loader_cls=TextLoader)

In [61]:
docs = loader.load()
len(docs)

100%|██████████| 2/2 [00:00<00:00, 308.80it/s]


2

## Splitter

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600,
    chunk_overlap=200,
    encoding_name='o200k_base'
)

In [63]:
splitted_docs = text_splitter.split_documents(docs)

len(splitted_docs)

11

## Embedding model

In [64]:
from langchain_openai import OpenAIEmbeddings

In [65]:
# 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 차원 크기를 계산
dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)

1536


## Vector DB - FAISS

In [66]:
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [67]:
db = FAISS.from_documents(
    documents=splitted_docs, embedding=embeddings)

In [68]:
db.index_to_docstore_id

{0: '2fa9041e-8b36-41b8-af27-bd959a485d99',
 1: '7fea7ef2-ee4e-4cc5-9237-642c4b4f0bce',
 2: '3d05f387-d5fc-497a-b38f-db6fb67a5036',
 3: '4825d60c-c81a-4f7d-804a-a65ac0d97dae',
 4: '87e6a9ef-628a-4d93-a018-e892efca3fb5',
 5: 'fd0e01b4-8bb4-4978-97d5-fb5a6a5f9163',
 6: '2c9ffa6b-737a-4121-883c-e22047a9b106',
 7: '3dd00830-f6f3-43a1-9c32-ac42c28494ee',
 8: '935f2324-a2f2-4988-8edb-1f1fa56c81a7',
 9: 'cb74c346-8835-47a8-85fd-0633b2bbf811',
 10: '11e12767-30c2-40ca-a2cf-4f918c3853ad'}

## 검색기로 변환 (as_retriever)
- as_retriever 메서드는 현재 벡터 저장소를 기반으로 VectorStoreRetriever 객체를 생성하는 기능을 제공합니다.

### 유사도

In [69]:
# 검색기로 변환
retriever = db.as_retriever()

In [70]:
# 검색 수행
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='2c9ffa6b-737a-4121-883c-e22047a9b106', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content='HuggingFace\n\n정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.\n예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.\n연관키워드: 혁신, 기술, 비즈니스 모델\n\nCrawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec\n\n정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large 

### MMR
- 다양성이 높은 더 많은 문서 검색
  - `k`: 반환할 문서 수 (기본값: 4)
  - `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
  - `lambda_mult`: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5, 0: 유사도 점수만 고려, 1: 다양성만 고려)

In [71]:
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
)

In [72]:
# 검색 수행
retriever.invoke("TF IDF 에 대하여 알려줘")

[Document(id='935f2324-a2f2-4988-8edb-1f1fa56c81a7', metadata={'source': '/content/data/MyDrive/ai_lecture/5. Large Language Models/data/vector_db/nlp-keywords.txt'}, page_content="TF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame\n\n정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.\n예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이터 조작과 분석을 용이하게 합니다.\n연관키워드: 데이터 분석, 판다스, 데이터 처리\n\nAttention 메커니즘\n\n정의: Attention 메커니즘은 딥러닝에서 중요한 정보에 